In [1]:
# Upgrade pip & install exactly pinned versions
!pip install --upgrade pip
!pip install numpy==1.26.0 \
            Pillow==9.4.0 \
            matplotlib==3.5.3 \
            protobuf==3.20.3 \
            lxml==4.9.2 \
            pycocotools==2.0.6 \
            absl-py==1.4.0 \
            kaggle==1.5.12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 56.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 206.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 129.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 164.6 MB/s eta 0:00:00
  DEPRECATION: Building 'matplotlib' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by set

In [1]:
%cd /content
!git clone --depth 1 https://github.com/tensorflow/models.git


/content
Cloning into 'models'...
remote: Enumerating objects: 4352, done.
remote: Counting objects: 100% (4352/4352), done.
remote: Compressing objects: 100% (3161/3161), done.
remote: Total 4352 (delta 1188), reused 3996 (delta 1118), pack-reused 0 (from 0)
Receiving objects: 100% (4352/4352), 54.72 MiB | 17.07 MiB/s, done.
Resolving deltas: 100% (1188/1188), done.


In [2]:
%cd /content/models/research/
!protoc object_detection/protos/*.proto --python_out=.

/content/models/research


In [3]:
!apt-get install -y protobuf-compiler


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
protobuf-compiler is already the newest version (3.12.4-1ubuntu7.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [4]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

In [5]:
import os
import sys

# Add the research and slim directories to the PYTHONPATH
sys.path.append('/content/models/research')
sys.path.append('/content/models/research/slim')

In [6]:
import tensorflow as tf
from object_detection.utils import config_util
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.18.0


In [7]:
!pip install kagglehub

In [8]:
import kagglehub

path = kagglehub.dataset_download("muzammilaliveltech/farm-harmful-animals-dataset")

print("Path to dataset files:", path)

100%|██████████| 810M/810M [00:09<00:00, 88.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1


In [9]:
import os

print("Files in dataset folder:")
print(os.listdir(path))


Files in dataset folder:
['Farm Harmful Animal Dataset']


In [10]:
dataset_dir = os.path.join(path, "Farm Harmful Animal Dataset")
print("Dataset directory:", dataset_dir)
print("Contents:")
print(os.listdir(dataset_dir))


Dataset directory: /root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset
Contents:
['train', 'test', 'validation']


In [11]:
train_dir = "/root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/train"
val_dir = "/root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/validation"
test_dir = "/root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/test"


In [12]:
import tensorflow as tf

IMG_SIZE = (224, 224)   # You can adjust to your model input size
BATCH_SIZE = 32

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    seed=42,
    shuffle=True,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    seed=42,
    shuffle=True,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    seed=42,
    shuffle=False,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)


Found 7262 files belonging to 15 classes.
Found 923 files belonging to 15 classes.
Found 899 files belonging to 15 classes.


In [13]:
for images, labels in train_dataset.take(1):
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)


Images shape: (32, 224, 224, 3)
Labels shape: (32,)


In [14]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2

# Get the number of classes automatically
num_classes = len(train_dataset.class_names)
print("Number of classes:", num_classes)

# Load MobileNetV2 base (without the top classification layers)
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,  # exclude the final dense layers
    weights="imagenet"  # use pre-trained weights
)

# Freeze the base model (so it doesn't train initially)
base_model.trainable = False

# Create the full model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(num_classes, activation='softmax')
])

model.summary()


Number of classes: 15
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15)             │        19,215 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,277,199 (8.69 MB)

 Trainable params: 19,215 (75.06 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [15]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [16]:
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.3),
    layers.RandomZoom(0.3),
    layers.RandomContrast(0.2),
    layers.RandomTranslation(0.1, 0.1)
])


In [17]:
from tensorflow.keras import Model, Input, regularizers
from tensorflow.keras.applications import MobileNetV2

IMG_SIZE = (224, 224)
num_classes = len(train_dataset.class_names)

inputs = Input(shape=(*IMG_SIZE, 3))

# Augmentation
x = data_augmentation(inputs)

# Preprocessing for MobileNetV2
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
x = preprocess_input(x)

# Base model
base_model = MobileNetV2(
    input_shape=(*IMG_SIZE, 3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False

x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.4)(x)  # More dropout for stronger regularization

# Dense layer with kernel regularizer
outputs = layers.Dense(
    num_classes,
    activation="softmax",
    kernel_regularizer=regularizers.l2(0.01)
)(x)

model = Model(inputs, outputs)


In [18]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [19]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=15,              # Wait longer before stopping
        restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        "best_model.h5",
        monitor="val_accuracy",
        save_best_only=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.2,
        patience=5,
        verbose=1,
        min_lr=1e-6
    )
]


In [20]:
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=100,
    callbacks=callbacks
)


Epoch 1/100
 41/227 ━━━━━━━━━━━━━━━━━━━━ 13s 72ms/step - accuracy: 0.0565 - loss: 3.7788

InvalidArgumentError: Graph execution error:

Detected at node decode_image/DecodeImage defined at (most recent call last):
<stack traces unavailable>
Detected at node decode_image/DecodeImage defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) INVALID_ARGUMENT:  Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]]
	 [[IteratorGetNext/_2]]
  (1) INVALID_ARGUMENT:  Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]]
0 successful operations.
0 derived errors ignored. [Op:__inference_multi_step_on_iterator_11519]

In [21]:
import os

train_dir = "/root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/train"

for root, dirs, files in os.walk(train_dir):
    for file in files:
        if not file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            print("❗ Non-image file:", os.path.join(root, file))


❗ Non-image file: /root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/train/Armadilles/Label/0143.txt
❗ Non-image file: /root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/train/Armadilles/Label/0544.txt
❗ Non-image file: /root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/train/Armadilles/Label/0180.txt
❗ Non-image file: /root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/train/Armadilles/Label/0035.txt
❗ Non-image file: /root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/train/Armadilles/Label/0020.txt
❗ Non-image file: /root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/train/Armadilles/L

In [22]:
import os

for root, dirs, files in os.walk(train_dir):
    for file in files:
        if not file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            file_path = os.path.join(root, file)
            print("Deleting:", file_path)
            os.remove(file_path)


Deleting: /root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/train/Armadilles/Label/0143.txt
Deleting: /root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/train/Armadilles/Label/0544.txt
Deleting: /root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/train/Armadilles/Label/0180.txt
Deleting: /root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/train/Armadilles/Label/0035.txt
Deleting: /root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/train/Armadilles/Label/0020.txt
Deleting: /root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/train/Armadilles/Label/0159.txt
Deleting: /root/.cache/kagglehub/d

In [23]:
# Validation directory
validation_dir = "/root/.../validation"
# Repeat cleaning logic here

# Test directory
test_dir = "/root/.../test"
# Repeat cleaning logic here


In [25]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)


Found 7262 files belonging to 15 classes.


NotFoundError: Could not find directory /root/.../validation

In [26]:
train_dir = "/root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/train"
validation_dir = "/root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/validation"
test_dir = "/root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/test"


In [27]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)


Found 7262 files belonging to 15 classes.
Found 923 files belonging to 15 classes.
Found 899 files belonging to 15 classes.


In [28]:
import os

print("Validation exists:", os.path.exists(validation_dir))
print("Test exists:", os.path.exists(test_dir))


Validation exists: True
Test exists: True


In [29]:
train_dir = "/root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/train"
validation_dir = "/root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/validation"
test_dir = "/root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/test"


In [30]:
import os

def clean_dir(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if not file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                file_path = os.path.join(root, file)
                print("Deleting:", file_path)
                os.remove(file_path)

# Clean all three
clean_dir(train_dir)
clean_dir(validation_dir)
clean_dir(test_dir)


Deleting: /root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/validation/Armadilles/Label/0513.txt
Deleting: /root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/validation/Armadilles/Label/0181.txt
Deleting: /root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/validation/Armadilles/Label/0067.txt
Deleting: /root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/validation/Armadilles/Label/0368.txt
Deleting: /root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/validation/Armadilles/Label/0505.txt
Deleting: /root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/validation/Armadilles/Label/0387.txt
Dele

In [31]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)


Found 7262 files belonging to 15 classes.
Found 923 files belonging to 15 classes.
Found 899 files belonging to 15 classes.


In [33]:
import os
import imghdr

# paths to your splits
train_dir      = "/root/.cache/kagglehub/datasets/.../Farm Harmful Animal Dataset/train"
validation_dir = "/root/.cache/kagglehub/datasets/.../Farm Harmful Animal Dataset/validation"
test_dir       = "/root/.cache/kagglehub/datasets/.../Farm Harmful Animal Dataset/test"

def clean_image_folder(folder):
    removed = 0
    for root, _, files in os.walk(folder):
        for fname in files:
            path = os.path.join(root, fname)
            kind = imghdr.what(path)  # returns 'jpeg','png','gif', etc, or None
            if kind not in ('jpeg','png','gif','bmp'):
                print(f"🗑️ Removing non‑image or corrupted file: {path}")
                os.remove(path)
                removed += 1
    print(f"Finished cleaning {folder!r}, removed {removed} files.\n")

for d in (train_dir, validation_dir, test_dir):
    clean_image_folder(d)


Finished cleaning '/root/.cache/kagglehub/datasets/.../Farm Harmful Animal Dataset/train', removed 0 files.

Finished cleaning '/root/.cache/kagglehub/datasets/.../Farm Harmful Animal Dataset/validation', removed 0 files.

Finished cleaning '/root/.cache/kagglehub/datasets/.../Farm Harmful Animal Dataset/test', removed 0 files.



/tmp/ipython-input-33-3771229057.py:2: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


In [35]:
import os

# ‘path’ is what you got back from kagglehub.dataset_download(...)
# It points to the folder containing “Farm Harmful Animal Dataset”
dataset_root = os.path.join(path, "Farm Harmful Animal Dataset")

train_dir      = os.path.join(dataset_root, "train")
validation_dir = os.path.join(dataset_root, "validation")
test_dir       = os.path.join(dataset_root, "test")

# Quick sanity‑check
print("Train exists:", os.path.isdir(train_dir))
print("Val   exists:", os.path.isdir(validation_dir))
print("Test  exists:", os.path.isdir(test_dir))


Train exists: True
Val   exists: True
Test  exists: True


In [36]:
import imghdr

def clean_image_folder(folder):
    removed = 0
    for root, _, files in os.walk(folder):
        for fname in files:
            full = os.path.join(root, fname)
            kind = imghdr.what(full)
            if kind not in ('jpeg','png','gif','bmp'):
                os.remove(full)
                removed += 1
    print(f"Cleaned {folder!r}: removed {removed} bad files")

for d in (train_dir, validation_dir, test_dir):
    clean_image_folder(d)


Cleaned '/root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/train': removed 109 bad files
Cleaned '/root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/validation': removed 23 bad files
Cleaned '/root/.cache/kagglehub/datasets/muzammilaliveltech/farm-harmful-animals-dataset/versions/1/Farm Harmful Animal Dataset/test': removed 18 bad files


In [37]:
import tensorflow as tf

IMG_SIZE   = (224, 224)
BATCH_SIZE = 32

train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)
test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)


Found 7153 files belonging to 15 classes.
Found 900 files belonging to 15 classes.
Found 881 files belonging to 15 classes.


In [38]:
# Install Keras Tuner
!pip install -q keras-tuner

import os
import tensorflow as tf
from tensorflow.keras import layers, regularizers, Model, Input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import kerastuner as kt

#─── 1) Dataset (assumes train_dataset, validation_dataset already re‑created) ───
IMG_SIZE   = (224, 224)
BATCH_SIZE = 32
# train_dataset, validation_dataset, test_dataset are assumed available

#─── 2) Data Augmentation ───────────────────────────────────────────────────────
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.3),
    layers.RandomZoom(0.3),
    layers.RandomContrast(0.2),
    layers.RandomTranslation(0.1, 0.1),
])

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
num_classes = len(train_dataset.class_names)

#─── 3) Model Builder for Tuner ────────────────────────────────────────────────
def build_model(hp):
    inputs = Input(shape=(*IMG_SIZE, 3))
    x = data_augmentation(inputs)
    x = preprocess_input(x)

    base_model = MobileNetV2(
        input_shape=(*IMG_SIZE, 3),
        include_top=False,
        weights="imagenet"
    )
    base_model.trainable = False
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)

    # Tune dropout rate
    dropout_rate = hp.Float("dropout", 0.2, 0.5, step=0.1)
    x = layers.Dropout(dropout_rate)(x)

    # Tune L2 regularization
    l2_reg = hp.Float("l2_reg", 1e-4, 1e-2, sampling="log")
    outputs = layers.Dense(
        num_classes,
        activation="softmax",
        kernel_regularizer=regularizers.l2(l2_reg)
    )(x)

    model = Model(inputs, outputs)

    # Tune learning rate
    lr = hp.Float("learning_rate", 1e-4, 1e-2, sampling="log")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

#─── 4) Run Hyperband Search ────────────────────────────────────────────────────
tuner = kt.Hyperband(
    build_model,
    objective="val_accuracy",
    max_epochs=20,
    factor=3,
    directory="kt_dir",
    project_name="animal10_tuning"
)

# Early stop within tuner
stop_early = EarlyStopping(monitor="val_accuracy", patience=5)

tuner.search(
    train_dataset,
    validation_data=validation_dataset,
    epochs=20,
    callbacks=[stop_early]
)

# Get the optimal hyperparameters
best_hp = tuner.get_best_hyperparameters(1)[0]
print("Best learning rate:", best_hp.get("learning_rate"))
print("Best dropout:",      best_hp.get("dropout"))
print("Best l2_reg:",       best_hp.get("l2_reg"))

#─── 5) Build & Train Final Model ──────────────────────────────────────────────
# Re‑build with best HP
model = build_model(best_hp)

# Callbacks for final training
callbacks = [
    EarlyStopping(monitor="val_accuracy", patience=15, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_accuracy", factor=0.5, patience=5, min_lr=1e-6, verbose=1),
    ModelCheckpoint("best_model_final.h5", monitor="val_accuracy", save_best_only=True)
]

# Final training: 100 epochs
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=100,
    callbacks=callbacks
)

#─── 6) Evaluate ───────────────────────────────────────────────────────────────
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Final Test Accuracy: {test_acc*100:.2f}%")


Trial 30 Complete [00h 11m 44s]
val_accuracy: 0.9588888883590698

Best val_accuracy So Far: 0.9622222185134888
Total elapsed time: 01h 52m 55s
Best learning rate: 0.0004645769617789411
Best dropout: 0.2
Best l2_reg: 0.0005355867575825687
Epoch 1/100
224/224 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.3795 - loss: 2.0760

224/224 ━━━━━━━━━━━━━━━━━━━━ 34s 112ms/step - accuracy: 0.3803 - loss: 2.0734 - val_accuracy: 0.8889 - val_loss: 0.5405 - learning_rate: 4.6458e-04
Epoch 2/100
223/224 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.7538 - loss: 0.8405

224/224 ━━━━━━━━━━━━━━━━━━━━ 23s 100ms/step - accuracy: 0.7539 - loss: 0.8401 - val_accuracy: 0.9211 - val_loss: 0.3507 - learning_rate: 4.6458e-04
Epoch 3/100
223/224 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.7903 - loss: 0.6935

224/224 ━━━━━━━━━━━━━━━━━━━━ 41s 101ms/step - accuracy: 0.7904 - loss: 0.6933 - val_accuracy: 0.9400 - val_loss: 0.2943 - learning_rate: 4.6458e-04
Epoch 4/100
223/224 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.8004 - loss: 0.6383

224/224 ━━━━━━━━━━━━━━━━━━━━ 25s 110ms/step - accuracy: 0.8004 - loss: 0.6381 - val_accuracy: 0.9411 - val_loss: 0.2703 - learning_rate: 4.6458e-04
Epoch 5/100
224/224 ━━━━━━━━━━━━━━━━━━━━ 38s 95ms/step - accuracy: 0.8171 - loss: 0.6073 - val_accuracy: 0.9389 - val_loss: 0.2566 - learning_rate: 4.6458e-04
Epoch 6/100
224/224 ━━━━━━━━━━━━━━━━━━━━ 22s 99ms/step - accuracy: 0.8269 - loss: 0.5573 - val_accuracy: 0.9389 - val_loss: 0.2533 - learning_rate: 4.6458e-04
Epoch 7/100
224/224 ━━━━━━━━━━━━━━━━━━━━ 43s 108ms/step - accuracy: 0.8303 - loss: 0.5564 - val_accuracy: 0.9411 - val_loss: 0.2364 - learning_rate: 4.6458e-04
Epoch 8/100
224/224 ━━━━━━━━━━━━━━━━━━━━ 39s 99ms/step - accuracy: 0.8441 - loss: 0.5199 - val_accuracy: 0.9411 - val_loss: 0.2374 - learning_rate: 4.6458e-04
Epoch 9/100
223/224 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.8453 - loss: 0.5161
Epoch 9: ReduceLROnPlateau reducing learning rate to 0.00023228848294820637.
224/224 ━━━━━━━━━━━━━━━━━━━━ 40s 96ms/step - accur

224/224 ━━━━━━━━━━━━━━━━━━━━ 23s 100ms/step - accuracy: 0.8488 - loss: 0.4902 - val_accuracy: 0.9467 - val_loss: 0.2240 - learning_rate: 2.3229e-04
Epoch 11/100
223/224 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.8570 - loss: 0.4860

224/224 ━━━━━━━━━━━━━━━━━━━━ 23s 103ms/step - accuracy: 0.8570 - loss: 0.4860 - val_accuracy: 0.9489 - val_loss: 0.2216 - learning_rate: 2.3229e-04
Epoch 12/100
224/224 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.8448 - loss: 0.4965

224/224 ━━━━━━━━━━━━━━━━━━━━ 23s 103ms/step - accuracy: 0.8448 - loss: 0.4964 - val_accuracy: 0.9500 - val_loss: 0.2207 - learning_rate: 2.3229e-04
Epoch 13/100
224/224 ━━━━━━━━━━━━━━━━━━━━ 39s 95ms/step - accuracy: 0.8525 - loss: 0.4878 - val_accuracy: 0.9489 - val_loss: 0.2187 - learning_rate: 2.3229e-04
Epoch 14/100
223/224 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.8434 - loss: 0.4924

224/224 ━━━━━━━━━━━━━━━━━━━━ 43s 103ms/step - accuracy: 0.8435 - loss: 0.4923 - val_accuracy: 0.9533 - val_loss: 0.2123 - learning_rate: 2.3229e-04
Epoch 15/100
224/224 ━━━━━━━━━━━━━━━━━━━━ 39s 94ms/step - accuracy: 0.8538 - loss: 0.4677 - val_accuracy: 0.9500 - val_loss: 0.2156 - learning_rate: 2.3229e-04
Epoch 16/100
224/224 ━━━━━━━━━━━━━━━━━━━━ 42s 99ms/step - accuracy: 0.8477 - loss: 0.4695 - val_accuracy: 0.9511 - val_loss: 0.2122 - learning_rate: 2.3229e-04
Epoch 17/100
224/224 ━━━━━━━━━━━━━━━━━━━━ 24s 108ms/step - accuracy: 0.8645 - loss: 0.4638 - val_accuracy: 0.9433 - val_loss: 0.2126 - learning_rate: 2.3229e-04
Epoch 18/100
224/224 ━━━━━━━━━━━━━━━━━━━━ 22s 99ms/step - accuracy: 0.8628 - loss: 0.4677 - val_accuracy: 0.9422 - val_loss: 0.2188 - learning_rate: 2.3229e-04
Epoch 19/100
223/224 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.8604 - loss: 0.4633
Epoch 19: ReduceLROnPlateau reducing learning rate to 0.00011614424147410318.
224/224 ━━━━━━━━━━━━━━━━━━━━ 41s 101ms/step 

In [39]:
# Unfreeze the base model
base_model.trainable = True

# It's often best to unfreeze only the top handful of layers:
# for layer in base_model.layers[:100]:
#     layer.trainable = False

# Re‑compile with lower LR
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-6),   # very small LR
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Fine‑tune
fine_tune_history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=50,           # another long run
    callbacks=callbacks
)


Epoch 1/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 42s 135ms/step - accuracy: 0.8482 - loss: 0.4831 - val_accuracy: 0.9533 - val_loss: 0.2120 - learning_rate: 1.0000e-06
Epoch 2/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 23s 101ms/step - accuracy: 0.8585 - loss: 0.4698 - val_accuracy: 0.9533 - val_loss: 0.2118 - learning_rate: 1.0000e-06
Epoch 3/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 24s 107ms/step - accuracy: 0.8632 - loss: 0.4525 - val_accuracy: 0.9533 - val_loss: 0.2116 - learning_rate: 1.0000e-06
Epoch 4/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 41s 106ms/step - accuracy: 0.8597 - loss: 0.4740 - val_accuracy: 0.9522 - val_loss: 0.2115 - learning_rate: 1.0000e-06
Epoch 5/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 40s 100ms/step - accuracy: 0.8541 - loss: 0.4653 - val_accuracy: 0.9511 - val_loss: 0.2115 - learning_rate: 1.0000e-06
Epoch 6/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 27s 123ms/step - accuracy: 0.8656 - loss: 0.4595 - val_accuracy: 0.9511 - val_loss: 0.2112 - learning_rate: 1.0000e-06
Epoch 7/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 36s 10

In [40]:
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy after fine‑tuning: {test_acc*100:.2f}%")


28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 86ms/step - accuracy: 0.9552 - loss: 0.1933
Test Accuracy after fine‑tuning: 94.44%


In [41]:
model.save("animal10_classifier.h5")
